In [1]:
import numpy as np
from PIL import Image, ImageTk, ImageFilter
import tkinter as tk
from tkinter import filedialog
# Tamanho máximo da imagem
MAX_WIDTH = 400  # Limite de largura da imagem
MAX_HEIGHT = 300  # Limite de altura da imagem

def carregar_imagem():
    """Função para carregar imagem e exibir na interface."""
    global img_original, img_exibicao
    caminho_imagem = filedialog.askopenfilename()
    if caminho_imagem:
        img_original = Image.open(caminho_imagem).convert('L')
        # Redimensiona a imagem para caber dentro do tamanho máximo
        img_original.thumbnail((MAX_WIDTH, MAX_HEIGHT))
        img_exibicao = ImageTk.PhotoImage(img_original)
        painel_imagem_original.config(image=img_exibicao)
        painel_imagem_original.image = img_exibicao

def filtro_passa_baixa():
    """Aplica filtro passa-baixa (Gaussian Blur) e exibe imagem filtrada."""
    if img_original is not None:
        img_filtrada = img_original.filter(ImageFilter.GaussianBlur(radius=7))
        exibir_imagem_filtrada(img_filtrada)

def filtro_media():
    """Aplica filtro passa-baixa (média) manualmente e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        kernel_size = 15
        kernel = np.ones((kernel_size, kernel_size)) / (kernel_size ** 2)
        img_filtrada = aplicar_filtro_movel(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_filtrada.astype(np.uint8))
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_mediana():
    """Aplica filtro passa-baixa (mediana) manualmente e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        kernel_size = 15
        img_filtrada = aplicar_filtro_mediana(img_array, kernel_size)
        img_filtrada_pil = Image.fromarray(img_filtrada.astype(np.uint8))
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_passa_alta():
    """Aplica filtro passa-alta (subtração de passa-baixa) e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        kernel_size = 5
        kernel = np.ones((kernel_size, kernel_size)) / (kernel_size ** 2)
        img_suave = aplicar_filtro_movel(img_array, kernel)
        img_filtrada = img_array.astype(np.float32) - img_suave.astype(np.float32)
        img_filtrada = np.clip(2.0 * img_filtrada, 0, 255).astype(np.uint8)
        img_filtrada_pil = Image.fromarray(img_filtrada)
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_laplaciano():
    """Aplica filtro passa-alta (Laplaciano) e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        laplacian_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
        img_filtrada = aplicar_filtro_movel(img_array, laplacian_kernel)
        img_filtrada = np.clip(img_filtrada, 0, 255).astype(np.uint8)
        img_filtrada_pil = Image.fromarray(img_filtrada)
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_sobel():
    """Aplica filtro Sobel (detecção de bordas) e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
        grad_x = aplicar_filtro_movel(img_array, sobel_x)
        grad_y = aplicar_filtro_movel(img_array, sobel_y)
        img_filtrada = np.hypot(grad_x, grad_y)
        img_filtrada = (img_filtrada / img_filtrada.max() * 255).astype(np.uint8)
        img_filtrada_pil = Image.fromarray(img_filtrada)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_filtro_movel(imagem, kernel):
    """Função genérica para aplicar a convolução com o kernel fornecido."""
    h, w = imagem.shape
    kh, kw = kernel.shape
    img_filtrada = np.zeros_like(imagem)
    for i in range(kh // 2, h - kh // 2):
        for j in range(kw // 2, w - kw // 2):
            img_filtrada[i, j] = np.sum(imagem[i - kh // 2:i + kh // 2 + 1, j - kw // 2:j + kw // 2 + 1] * kernel)
    return img_filtrada

def aplicar_filtro_mediana(imagem, kernel_size):
    """Aplica filtro de mediana manualmente."""
    h, w = imagem.shape
    img_filtrada = np.zeros_like(imagem)
    for i in range(kernel_size // 2, h - kernel_size // 2):
        for j in range(kernel_size // 2, w - kernel_size // 2):
            patch = imagem[i - kernel_size // 2:i + kernel_size // 2 + 1, j - kernel_size // 2:j + kernel_size // 2 + 1]
            img_filtrada[i, j] = np.median(patch)
    return img_filtrada

# Função de Erosão
def erosao(imagem, kernel):
    img_erodida = np.zeros_like(imagem)
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    altura, largura = imagem.shape
    
    # Percorrer todos os pixels da imagem
    for i in range(pad_h, altura - pad_h):
        for j in range(pad_w, largura - pad_w):
            # Obtenção da região onde o kernel vai atuar
            region = imagem[i - pad_h:i + pad_h + 1, j - pad_w:j + pad_w + 1]
            # Aplicar a operação de erosão (mínimo dos valores multiplicados pelo kernel)
            img_erodida[i, j] = np.min(region * kernel)
    return img_erodida

# Função de Dilatação
def dilatacao(imagem, kernel):
    img_dilatada = np.zeros_like(imagem)
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    altura, largura = imagem.shape
    
    # Percorrer todos os pixels da imagem
    for i in range(pad_h, altura - pad_h):
        for j in range(pad_w, largura - pad_w):
            # Obtenção da região onde o kernel vai atuar
            region = imagem[i - pad_h:i + pad_h + 1, j - pad_w:j + pad_w + 1]
            # Aplicar a operação de dilatação (máximo dos valores multiplicados pelo kernel)
            img_dilatada[i, j] = np.max(region * kernel)
    return img_dilatada

# Função de Abertura (Erosão seguida de Dilatação)
def abertura(imagem, kernel):
    imagem_erosionada = erosao(imagem, kernel)
    imagem_aberta = dilatacao(imagem_erosionada, kernel)
    return imagem_aberta

# Função de Fechamento (Dilatação seguida de Erosão)
def fechamento(imagem, kernel):
    imagem_dilatada = dilatacao(imagem, kernel)
    imagem_fechada = erosao(imagem_dilatada, kernel)
    return imagem_fechada

# Função de Exibição de Imagem (para integração com sua interface)
def exibir_imagem_filtrada(img_filtrada_pil):
    img_filtrada_pil.show()

# Funções de Aplicação (para serem chamadas de seu código de interface)

def aplicar_erosao():
    if img_original is not None:
        img_array = np.array(img_original.convert('L'))  # Convertendo para escala de cinza
        kernel = np.ones((5, 5), np.uint8)
        img_erodida = erosao(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_erodida)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_dilatacao():
    if img_original is not None:
        img_array = np.array(img_original.convert('L'))  # Convertendo para escala de cinza
        kernel = np.ones((5, 5), np.uint8)
        img_dilatada = dilatacao(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_dilatada)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_abertura():
    if img_original is not None:
        img_array = np.array(img_original.convert('L'))  # Convertendo para escala de cinza
        kernel = np.ones((5, 5), np.uint8)
        img_aberta = abertura(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_aberta)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_fechamento():
    if img_original is not None:
        img_array = np.array(img_original.convert('L'))  # Convertendo para escala de cinza
        kernel = np.ones((5, 5), np.uint8)
        img_fechada = fechamento(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_fechada)
        exibir_imagem_filtrada(img_filtrada_pil)


def aplicar_limiarizacao():
    if img_original is not None:
        # Converte a imagem original para escala de cinza
        img_array = np.array(img_original.convert('L'))  # 'L' converte para escala de cinza

        # Cria uma nova imagem vazia para armazenar o resultado da limiarização
        img_thresh = np.zeros_like(img_array)

        # Aplica a limiarização manualmente pixel por pixel
        for i in range(img_array.shape[0]):  # Percorre as linhas
            for j in range(img_array.shape[1]):  # Percorre as colunas
                if img_array[i, j] > 128:
                    img_thresh[i, j] = 255  # Valor branco
                else:
                    img_thresh[i, j] = 0  # Valor preto

        # Converte o resultado de volta para uma imagem PIL
        img_filtrada_pil = Image.fromarray(img_thresh)
        
        # Exibe a imagem filtrada
        exibir_imagem_filtrada(img_filtrada_pil)


def aplicar_limiarizacao_adaptativa():
    if img_original is not None:
        img_array = np.array(img_original)
        img_thresh_adapt = limiarizacao_adaptativa(img_array)
        img_filtrada_pil = Image.fromarray(img_thresh_adapt)
        exibir_imagem_filtrada(img_filtrada_pil)


def limiarizacao_adaptativa(imagem, kernel_size=11, C=2):
    img_filtrada = np.zeros_like(imagem)
    pad_size = kernel_size // 2
    imagem_padded = np.pad(imagem, pad_size, mode='reflect')
    for i in range(pad_size, imagem_padded.shape[0] - pad_size):
        for j in range(pad_size, imagem_padded.shape[1] - pad_size):
            local_region = imagem_padded[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
            local_mean = np.mean(local_region)
            img_filtrada[i - pad_size, j - pad_size] = 255 if imagem[i - pad_size, j - pad_size] > local_mean - C else 0
    return img_filtrada


def exibir_imagem_filtrada(img_filtrada):
    """Exibe a imagem filtrada na interface."""
    img_exibicao_filtrada = ImageTk.PhotoImage(img_filtrada)
    painel_imagem_filtrada.config(image=img_exibicao_filtrada)
    painel_imagem_filtrada.image = img_exibicao_filtrada

def criar_interface():
    """Configura a interface gráfica com botões e painel de exibição."""
    global janela, painel_imagem_original, painel_imagem_filtrada
    janela = tk.Tk()
    janela.title("Editor de Fotos - Filtros de Imagem")
    painel_imagem_original = tk.Label(janela)
    painel_imagem_original.grid(row=0, column=0)
    painel_imagem_filtrada = tk.Label(janela)
    painel_imagem_filtrada.grid(row=0, column=1)
    criar_botoes()
    janela.mainloop()

import tkinter as tk

def criar_botoes():
    """Cria os botões para carregar imagem e aplicar filtros, organizados em grupos."""

    # Grupo para Carregar Imagem
    frame_carregar = tk.LabelFrame(janela, text="Carregar Imagem", padx=10, pady=10)
    frame_carregar.grid(row=1, column=0, padx=10, pady=10, sticky="w")
    botao_carregar = tk.Button(frame_carregar, text="Carregar Imagem", command=carregar_imagem)
    botao_carregar.pack()

    # Grupo para Filtros de Imagem
    frame_filtros = tk.LabelFrame(janela, text="Filtros de Imagem", padx=10, pady=10)
    frame_filtros.grid(row=1, column=2, padx=10, pady=10, sticky="w")
    botao_filtro_passa_baixa = tk.Button(frame_filtros, text="Filtro Passa-Baixa (Gaussian)", command=filtro_passa_baixa)
    botao_filtro_passa_baixa.grid(row=0, column=0)
    botao_filtro_media = tk.Button(frame_filtros, text="Filtro Passa-Baixa (Média)", command=filtro_media)
    botao_filtro_media.grid(row=0, column=1)
    botao_filtro_mediana = tk.Button(frame_filtros, text="Filtro Passa-Baixa (Mediana)", command=filtro_mediana)
    botao_filtro_mediana.grid(row=0, column=2)
    botao_filtro_passa_alta = tk.Button(frame_filtros, text="Filtro Passa-Alta (Subtração de Passa-Baixa)", command=filtro_passa_alta)
    botao_filtro_passa_alta.grid(row=1, column=0)
    botao_filtro_laplaciano = tk.Button(frame_filtros, text="Filtro Passa-Alta (Laplaciano)", command=filtro_laplaciano)
    botao_filtro_laplaciano.grid(row=1, column=1)
    botao_filtro_sobel = tk.Button(frame_filtros, text="Filtro Passa-Alta (Sobel)", command=filtro_sobel)
    botao_filtro_sobel.grid(row=1, column=2)

    # Grupo para Operações Morfológicas
    frame_morfologia = tk.LabelFrame(janela, text="Operações Morfológicas", padx=10, pady=10)
    frame_morfologia.grid(row=2, column=2, padx=10, pady=10, sticky="w")
    botao_erosao = tk.Button(frame_morfologia, text="Erosão", command=aplicar_erosao)
    botao_erosao.grid(row=0, column=0)
    botao_dilatacao = tk.Button(frame_morfologia, text="Dilatação", command=aplicar_dilatacao)
    botao_dilatacao.grid(row=0, column=1)
    botao_abertura = tk.Button(frame_morfologia, text="Abertura", command=aplicar_abertura)
    botao_abertura.grid(row=1, column=0)
    botao_fechamento = tk.Button(frame_morfologia, text="Fechamento", command=aplicar_fechamento)
    botao_fechamento.grid(row=1, column=1)

    # Grupo para Segmentação de Imagem
    frame_segmentacao = tk.LabelFrame(janela, text="Segmentação de Imagem", padx=10, pady=10)
    frame_segmentacao.grid(row=2, column=1, padx=10, pady=10, sticky="w")
    botao_limiarizacao = tk.Button(frame_segmentacao, text="Limiarização", command=aplicar_limiarizacao)
    botao_limiarizacao.grid(row=0, column=0)
    botao_limiarizacao_adaptativa = tk.Button(frame_segmentacao, text="Limiarização Adaptativa", command=aplicar_limiarizacao_adaptativa)
    botao_limiarizacao_adaptativa.grid(row=0, column=1)

# Executa o programa
if __name__ == "__main__":
    criar_interface()
